# Import Statements

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
import pandas as pd 
import os 

# Create the RFCModel class which holds the model, dataframe, and the teams of each league with their respective winrates  

In [72]:
class RFCModel():
    
    def __init__(self, league):
        self.clf = None 
        self.model_accuracy = None 
        self.league = league
        self.teams = {}
        self.df = None 
        self.df_no_teams = None 
        self.sorted_keys = None 
        
    def calculate_rankings(self):
        
        teams = self.df['team'].unique()
        for team in teams:
            # team_all_matches = pd.DataFrame(columns=self.df_no_teams.columns)
            # team_all_matches.columns = team_all_matches.columns.astype(str) 
            wins, losses = 0, 0
            
            for index, row in self.df.iterrows():
                if row['team'] == team:
                    new_row = (self.df_no_teams.iloc[index].drop(labels=['result']).values.reshape(1,-1))
                    predictions = self.clf.predict(new_row)
                    if predictions:
                        wins += 1 
                    else:
                        losses += 1 
            self.teams[team] = wins / (wins + losses) * 100 

        
    def create_model(self):
        processed_data_directory = './data_by_league'
        file_name = self.league + ".csv"
        file_path  = os.path.join(processed_data_directory, file_name)
        self.df = pd.read_csv(file_path)
        self.df.dropna(inplace=True, ignore_index=True)
        self.df_no_teams = self.df.drop(columns=['team', 'elder_dragon_kills', 'game_length', 'second_turret_kill', 'third_turret_kill', 'first_blood', 'file_name'], axis=1)
        
        x = self.df_no_teams.drop(columns=['result'])
        y = self.df_no_teams['result']
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        # print(f"training data: {X_train}")
        
        # Create a Random Forest Classifier
        self.clf = RandomForestClassifier(random_state=42)
        
        # Train the model
        self.clf.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = self.clf.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        
        self.model_accuracy = accuracy

    def sort_keys(self):
        self.sorted_keys = sorted(self.teams, key=lambda x: self.teams[x], reverse=True)
        

# Code to assign points to each team for ranking 

In [73]:
def assign_points(teams_dict, sorted_keys, league): 
    returning_dict = {}
    curr_score = 10 
    
    # Arbitrary additional scores based on perceived strength of the entire region 
    if league == 'LPL':
        curr_score += 4 
    elif league == 'LCK':
        curr_score += 3
    elif league == 'LEC':
        curr_score += 2 
    elif league == 'LCS':
        curr_score += 1 
    
    
    prev_win_rate = teams_dict[list(teams_dict.keys())[0]] 
    for team in sorted_keys:
        win_rate = teams_dict[team]        
        if prev_win_rate == win_rate:
            returning_dict[team] = curr_score

        else:
            prev_win_rate = win_rate 
            curr_score -= 1     
            returning_dict[team] = curr_score
            
    return returning_dict

In [74]:
lck = RFCModel("LCK")


# Calculate the points for all teams  

In [75]:
# processed_data_directory = './data_by_league'
# files = os.listdir(processed_data_directory)
# 
# for file_path in files:
#     file_name = os.path.join(processed_data_directory, file_path)
#     df = pd.read_csv(file_name)
 
 
all_team_scores = {}
leagues = ["LCS", "LEC", "CBLOL", "LCK", "LJL", "LLA", "LPL", "PCS", "VCS"]

for league in leagues:
    league_model = RFCModel(league)
    league_model.create_model()
    league_model.calculate_rankings()
    league_model.sort_keys()
    
    league_dict = assign_points(league_model.teams, league_model.sorted_keys, league)
    
    all_team_scores.update(league_dict)
    
all_team_scores
    
# a = "LCS.csv"
# a[:-4]

/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomFore

{'C9': 11,
 'TSM': 11,
 '100': 11,
 'IMT': 11,
 'FLY': 11,
 'EG': 10,
 'GG': 9,
 'DIG': 8,
 'NRG': 7,
 'TL': 6,
 'FNC': 11,
 'AST': 11,
 'VIT': 11,
 'G2': 10,
 'TH': 9,
 'BDS': 8,
 'MAD': 7,
 'XL': 6,
 'KOI': 5,
 'SK': 4,
 'LOS': 9,
 'KBM': 9,
 'RED': 8,
 'VKS': 7,
 'PNG': 6,
 'LLL': 5,
 'FX': 4,
 'ITZ': 3,
 'FUR': 2,
 'LBR': 1,
 'KT': 13,
 'GEN': 13,
 'KDF': 13,
 'DK': 12,
 'NS': 11,
 'HLE': 10,
 'T1': 9,
 'BRO': 8,
 'DRX': 7,
 'LSB': 6,
 'SG': 9,
 'BC': 9,
 'CGA': 9,
 'SHG': 8,
 'FL': 7,
 'DFM': 6,
 'AXZ': 5,
 'V3': 4,
 'R7': 9,
 'AZE': 8,
 'INF': 7,
 'ISG': 7,
 'EST': 6,
 '6K': 5,
 'AK': 4,
 'LEV': 3,
 'TES': 13,
 'UP': 13,
 'AL': 13,
 'JDG': 12,
 'LNG': 11,
 'WE': 10,
 'NIP': 10,
 'TT': 9,
 'OMG': 8,
 'RA': 7,
 'BLG': 6,
 'WBG': 5,
 'FPX': 4,
 'RNG': 3,
 'IG': 2,
 'EDG': 1,
 'LGD': 0,
 'HPS': 9,
 'WP': 9,
 'IMP': 9,
 'BYG': 8,
 'PSG': 7,
 'JT': 6,
 'CFO': 5,
 'FAK': 4,
 'DCG': 3,
 'DWT': 2,
 'SGB': 9,
 'TS': 8,
 'GAM': 7,
 'CES': 6,
 'TW': 5,
 'TF': 4,
 'MBE': 3,
 'SE': 2}

# The final list of teams, ranked best to worst 

In [76]:
all_team_scores = sorted(all_team_scores, key=lambda x: all_team_scores[x], reverse=True)
all_team_scores

['KT',
 'GEN',
 'KDF',
 'TES',
 'UP',
 'AL',
 'DK',
 'JDG',
 'C9',
 'TSM',
 '100',
 'IMT',
 'FLY',
 'FNC',
 'AST',
 'VIT',
 'NS',
 'LNG',
 'EG',
 'G2',
 'HLE',
 'WE',
 'NIP',
 'GG',
 'TH',
 'LOS',
 'KBM',
 'T1',
 'SG',
 'BC',
 'CGA',
 'R7',
 'TT',
 'HPS',
 'WP',
 'IMP',
 'SGB',
 'DIG',
 'BDS',
 'RED',
 'BRO',
 'SHG',
 'AZE',
 'OMG',
 'BYG',
 'TS',
 'NRG',
 'MAD',
 'VKS',
 'DRX',
 'FL',
 'INF',
 'ISG',
 'RA',
 'PSG',
 'GAM',
 'TL',
 'XL',
 'PNG',
 'LSB',
 'DFM',
 'EST',
 'BLG',
 'JT',
 'CES',
 'KOI',
 'LLL',
 'AXZ',
 '6K',
 'WBG',
 'CFO',
 'TW',
 'SK',
 'FX',
 'V3',
 'AK',
 'FPX',
 'FAK',
 'TF',
 'ITZ',
 'LEV',
 'RNG',
 'DCG',
 'MBE',
 'FUR',
 'IG',
 'DWT',
 'SE',
 'LBR',
 'EDG',
 'LGD']